# Imports

In [2]:
import json
import pandas as pd
import numpy as np
import sqlalchemy as db
import statsmodels.api as sm

from sodapy import Socrata

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Data Scraping

### Data: DHS Daily Report
- Source: https://data.cityofnewyork.us/Social-Services/DHS-Daily-Report/k46n-sa2m

This dataset includes the daily number of individuals residing in the Department of Homeless Services (DHS) shelter system and the daily number of families applying to the DHS shelter system.

---

### 1. Authorization

- **Step 1** Save your credential as following in a json file

```
{'app_token': 'Enter your app token here'}```

- **Step 2** Name the json file "creds.json" and save it in the directory this jupyter notebook is located.

In [3]:
# Read "creds.json"
creds_file = open('./creds.json', 'r')
socrata_creds = json.loads(creds_file.read())

In [4]:
# Sign in using your app token
client = Socrata("data.cityofnewyork.us", socrata_creds['app_token'])

### 2. Return the dataset as Pandas Dataframe
Use Socrata Open Data API (SODA) to return the dataset as dictionary and convert it into Pandas Dataframe.
SODA provides programmatic access to NYC OpenData datasets.
- Source: https://dev.socrata.com/foundry/data.cityofnewyork.us/k46n-sa2m

In [5]:
# Get the DHS Daily Report dataset using "Dataset Identifier"
results = client.get("k46n-sa2m", limit = 10_000);

# Convert to pandas DataFrame
daily_df = pd.DataFrame.from_records(results);

# Data Wrangling

In [7]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 13 columns):
date_of_census                                             2230 non-null object
total_adults_in_shelter                                    2230 non-null object
total_children_in_shelter                                  2230 non-null object
total_individuals_in_shelter                               2230 non-null object
single_adult_men_in_shelter                                2230 non-null object
single_adult_women_in_shelter                              2230 non-null object
total_single_adults_in_shelter                             2230 non-null object
families_with_children_in_shelter                          2230 non-null object
adults_in_families_with_children_in_shelter                2230 non-null object
children_in_families_with_children_in_shelter              2230 non-null object
total_individuals_in_families_with_children_in_shelter_    2230 non-null object
adult_familie

- The type of all columns is 'object'. Convert `date_of_census` column type to datetime and rest of the columns type to numerical in order to make timeseries analysis easier.
- No null value observed.

In [8]:
# Convert `data_of_census` column type to datetime and set the column as index.
daily_df['date_of_census'] = daily_df['date_of_census'].apply(lambda x: x.split('T')[0])
daily_df.index = pd.to_datetime(daily_df['date_of_census'], format = "%Y/%m/%d")
daily_df = daily_df.drop('date_of_census', axis = 1)

In [9]:
# Convert the type of all columns to numeric
daily_df = daily_df.apply(pd.to_numeric)

In [10]:
# Check and see the data cleaning is correctly executed.
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2230 entries, 2019-11-30 to 2013-08-21
Data columns (total 12 columns):
total_adults_in_shelter                                    2230 non-null int64
total_children_in_shelter                                  2230 non-null int64
total_individuals_in_shelter                               2230 non-null int64
single_adult_men_in_shelter                                2230 non-null int64
single_adult_women_in_shelter                              2230 non-null int64
total_single_adults_in_shelter                             2230 non-null int64
families_with_children_in_shelter                          2230 non-null int64
adults_in_families_with_children_in_shelter                2230 non-null int64
children_in_families_with_children_in_shelter              2230 non-null int64
total_individuals_in_families_with_children_in_shelter_    2230 non-null int64
adult_families_in_shelter                                  2230 non-null int64
indivi

- The index of the dataset is now 'DatetimeIndex' and all columns are numeric.
- No null vlaue

---

In [11]:
daily_df.head()

,total_adults_in_shelter,total_children_in_shelter,total_individuals_in_shelter,single_adult_men_in_shelter,single_adult_women_in_shelter,total_single_adults_in_shelter,families_with_children_in_shelter,adults_in_families_with_children_in_shelter,children_in_families_with_children_in_shelter,total_individuals_in_families_with_children_in_shelter_,adult_families_in_shelter,individuals_in_adult_families_in_shelter
date_of_census,,,,,,,,,,,,
2019-11-30,38214,21695,59909,12117,4565,16682,12188,16302,21695,37997,2486,5230
2019-11-29,38152,21662,59814,12062,4559,16621,12171,16277,21662,37939,2497,5254
2019-11-28,38168,21706,59874,12022,4587,16609,12197,16308,21706,38014,2496,5251
2019-11-27,38249,21698,59947,12130,4591,16721,12189,16301,21698,37999,2483,5227
2019-11-26,38388,21728,60116,12213,4616,16829,12198,16321,21728,38049,2488,5238


In [12]:
daily_df.resample('W').mean().head()

,total_adults_in_shelter,total_children_in_shelter,total_individuals_in_shelter,single_adult_men_in_shelter,single_adult_women_in_shelter,total_single_adults_in_shelter,families_with_children_in_shelter,adults_in_families_with_children_in_shelter,children_in_families_with_children_in_shelter,total_individuals_in_families_with_children_in_shelter_,adult_families_in_shelter,individuals_in_adult_families_in_shelter
date_of_census,,,,,,,,,,,,
2013-08-25,28342.800000,21334.400000,49677.200000,7184.200000,2698.200000,9882.400000,10283.200000,14636.800000,21334.400000,35971.200000,1801.200000,3823.600000
2013-09-01,28327.285714,21361.714286,49689.000000,7117.000000,2690.428571,9807.428571,10296.857143,14671.857143,21361.714286,36033.571429,1811.428571,3848.000000
2013-09-08,28381.000000,21448.600000,49829.600000,7100.800000,2698.000000,9798.800000,10331.800000,14722.000000,21448.600000,36170.600000,1818.000000,3860.200000
2013-09-15,28597.285714,21641.428571,50238.714286,7144.000000,2747.714286,9891.714286,10414.857143,14836.428571,21641.428571,36477.857143,1823.142857,3869.142857
2013-09-22,28729.285714,21694.000000,50423.285714,7213.285714,2771.285714,9984.571429,10445.714286,14879.857143,21694.000000,36573.857143,1821.428571,3864.857143


In [14]:
daily_df.resample('W').mean().isnull().sum()

total_adults_in_shelter                                    0
total_children_in_shelter                                  0
total_individuals_in_shelter                               0
single_adult_men_in_shelter                                0
single_adult_women_in_shelter                              0
total_single_adults_in_shelter                             0
families_with_children_in_shelter                          0
adults_in_families_with_children_in_shelter                0
children_in_families_with_children_in_shelter              0
total_individuals_in_families_with_children_in_shelter_    0
adult_families_in_shelter                                  0
individuals_in_adult_families_in_shelter                   0
dtype: int64

In [15]:
weekly_df = daily_df.resample('W').mean()

In [16]:
weekly_df.index

DatetimeIndex(['2013-08-25', '2013-09-01', '2013-09-08', '2013-09-15',
               '2013-09-22', '2013-09-29', '2013-10-06', '2013-10-13',
               '2013-10-20', '2013-10-27',
               ...
               '2019-09-29', '2019-10-06', '2019-10-13', '2019-10-20',
               '2019-10-27', '2019-11-03', '2019-11-10', '2019-11-17',
               '2019-11-24', '2019-12-01'],
              dtype='datetime64[ns]', name='date_of_census', length=328, freq='W-SUN')

# Database Contruction

PostgreSQL database was contructed using Heroku.

- Source: https://www.heroku.com/postgres

---
### Sign in

- **Step 1** Save your PostgreSQL URL as following in a json file

```
{"url":"Enter your PostgreSQL URL here"}
```

- **Step 2** Name the json file "database_creds.json" and save it in the directory this jupyter notebook is located.

In [18]:
# Load your PostgreSQL credential to `database_creds`
database_creds_file = open('./database_cred.json', 'r')
database_creds = json.loads(database_creds_file.read())

In [21]:
# Sign into your database
engine = db.create_engine(database_creds['url'])

connection = engine.connect()
metadata = db.MetaData()

### Create table `ny_dhs_weekly`

In [22]:
sql = f"""CREATE TABLE ny_dhs_weekly (date_of_census DATE"""
for col in list(results[0].keys())[1:]:
    sql += f", {col} INT"
sql += ")"

connection.execute(sql)

### Insert values in 'weekly_df' dataframe to `ny_dhs_weekly` table 

In [89]:
sql = "INSERT INTO ny_dhs_weekly VALUES"
for n_row in range(len(weekly_df)):
    sql += f" ('{str(weekly_df.index[n_row].date())}'"
    for n_col in range(len(weekly_df.columns)):
        sql += f", {int(weekly_df.iloc[n_row,n_col])}"
    sql += "),"

connection.execute(sql[:-1])

### Query `ny_dhs_weekly` to see if the database is correctly constructed.

In [90]:
sql = """
SELECT *
FROM ny_dhs_weekly
"""
connection.execute(sql)

In [99]:
database_df = pd.read_sql_query(sql, engine)

In [100]:
database_df.head()

,date_of_census,total_adults_in_shelter,total_children_in_shelter,total_individuals_in_shelter,single_adult_men_in_shelter,single_adult_women_in_shelter,total_single_adults_in_shelter,families_with_children_in_shelter,adults_in_families_with_children_in_shelter,children_in_families_with_children_in_shelter,total_individuals_in_families_with_children_in_shelter_,adult_families_in_shelter,individuals_in_adult_families_in_shelter
0,2013-08-25,28342,21334,49677,7184,2698,9882,10283,14636,21334,35971,1801,3823
1,2013-09-01,28327,21361,49689,7117,2690,9807,10296,14671,21361,36033,1811,3848
2,2013-09-08,28381,21448,49829,7100,2698,9798,10331,14722,21448,36170,1818,3860
3,2013-09-15,28597,21641,50238,7144,2747,9891,10414,14836,21641,36477,1823,3869
4,2013-09-22,28729,21694,50423,7213,2771,9984,10445,14879,21694,36573,1821,3864


In [101]:
database_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 13 columns):
date_of_census                                             328 non-null object
total_adults_in_shelter                                    328 non-null int64
total_children_in_shelter                                  328 non-null int64
total_individuals_in_shelter                               328 non-null int64
single_adult_men_in_shelter                                328 non-null int64
single_adult_women_in_shelter                              328 non-null int64
total_single_adults_in_shelter                             328 non-null int64
families_with_children_in_shelter                          328 non-null int64
adults_in_families_with_children_in_shelter                328 non-null int64
children_in_families_with_children_in_shelter              328 non-null int64
total_individuals_in_families_with_children_in_shelter_    328 non-null int64
adult_families_in_shelter           